In [32]:
import pandas as pd

In [6]:
import sys
sys.path.append("/Users/gabriel/Documents/Git/End-to-end MLOps for Time Series")
from utils import load_config
config = load_config("../config/config.yaml")

In [34]:
consumption_train = pd.read_csv("../data/preprocessed/consumption_train.csv", parse_dates=["datetime"])
print(consumption_train.shape)
consumption_train.head()

(760650, 9)


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,96.590,1,2021-09-01,0,1,0
1,3,0,3,39.241,1,2021-09-01,0,25,12
2,7,1,3,453.023,1,2021-09-01,0,61,30
3,8,0,1,9.787,1,2021-09-01,0,63,31
4,3,0,1,14.964,1,2021-09-01,0,23,11


# Lag features

In [36]:
def make_lags(df, column, lags, dropna=True):
    df_lags = pd.concat(
        {
            f'{column}_lag_{i}': df[column].shift(i)
            for i in lags
        },
        axis=1)
    if dropna:
        return pd.concat([df, df_lags], axis=1).dropna(axis=0)
    else:
        return pd.concat([df, df_lags], axis=1)

In [37]:
start = config["forecast_horizon"]
end = start + config["n_lags"] + 1
range(start, end)

range(48, 73)

In [38]:
groups = consumption_train.groupby(by="prediction_unit_id")
transformed_groups = []
for name, group in groups:
    group = make_lags(df=group, column="target", lags=range(start, end))
    transformed_groups.append(group)

In [39]:
new_consumption_train = pd.concat(transformed_groups, axis=0)
new_consumption_train.isna().sum().sum()

np.int64(0)

In [40]:
new_consumption_train.reset_index(drop=True, inplace=True)
new_consumption_train.index

RangeIndex(start=0, stop=756114, step=1)

In [41]:
new_consumption_train.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,target_lag_48,...,target_lag_63,target_lag_64,target_lag_65,target_lag_66,target_lag_67,target_lag_68,target_lag_69,target_lag_70,target_lag_71,target_lag_72
0,0,0,1,121.942,1,2021-09-04 00:00:00,3,8785,0,109.366,...,54.211,77.308,94.592,96.481,89.781,88.184,87.955,91.594,77.691,96.590
1,0,0,1,114.038,1,2021-09-04 01:00:00,3,8907,0,97.033,...,36.071,54.211,77.308,94.592,96.481,89.781,88.184,87.955,91.594,77.691
2,0,0,1,105.102,1,2021-09-04 02:00:00,3,9029,0,104.854,...,31.147,36.071,54.211,77.308,94.592,96.481,89.781,88.184,87.955,91.594
3,0,0,1,120.120,1,2021-09-04 03:00:00,3,9151,0,103.433,...,26.138,31.147,36.071,54.211,77.308,94.592,96.481,89.781,88.184,87.955
4,0,0,1,117.384,1,2021-09-04 04:00:00,3,9273,0,105.858,...,37.784,26.138,31.147,36.071,54.211,77.308,94.592,96.481,89.781,88.184


# Time-related features

In [42]:
def make_time_features(df, datetime_column):
    index = pd.DatetimeIndex(df[datetime_column])
    time_features = pd.DataFrame(index=index, data={"hour": index.hour, "dayofweek": index.dayofweek, "month": index.month})
    time_features.reset_index(drop=True, inplace=True)
    return pd.concat([df, time_features], axis=1)

In [43]:
new_consumption_train = make_time_features(df=new_consumption_train, datetime_column="datetime")
print(new_consumption_train.shape)
new_consumption_train.head()

(756114, 37)


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,target_lag_48,...,target_lag_66,target_lag_67,target_lag_68,target_lag_69,target_lag_70,target_lag_71,target_lag_72,hour,dayofweek,month
0,0,0,1,121.942,1,2021-09-04 00:00:00,3,8785,0,109.366,...,96.481,89.781,88.184,87.955,91.594,77.691,96.590,0,5,9
1,0,0,1,114.038,1,2021-09-04 01:00:00,3,8907,0,97.033,...,94.592,96.481,89.781,88.184,87.955,91.594,77.691,1,5,9
2,0,0,1,105.102,1,2021-09-04 02:00:00,3,9029,0,104.854,...,77.308,94.592,96.481,89.781,88.184,87.955,91.594,2,5,9
3,0,0,1,120.120,1,2021-09-04 03:00:00,3,9151,0,103.433,...,54.211,77.308,94.592,96.481,89.781,88.184,87.955,3,5,9
4,0,0,1,117.384,1,2021-09-04 04:00:00,3,9273,0,105.858,...,36.071,54.211,77.308,94.592,96.481,89.781,88.184,4,5,9


In [44]:
time_index = pd.DatetimeIndex(new_consumption_train["datetime"])
time_index

DatetimeIndex(['2021-09-04 00:00:00', '2021-09-04 01:00:00',
               '2021-09-04 02:00:00', '2021-09-04 03:00:00',
               '2021-09-04 04:00:00', '2021-09-04 05:00:00',
               '2021-09-04 06:00:00', '2021-09-04 07:00:00',
               '2021-09-04 08:00:00', '2021-09-04 09:00:00',
               ...
               '2023-01-25 20:00:00', '2023-01-25 21:00:00',
               '2023-01-25 22:00:00', '2023-01-25 23:00:00',
               '2023-01-26 00:00:00', '2023-01-26 01:00:00',
               '2023-01-26 02:00:00', '2023-01-26 03:00:00',
               '2023-01-26 04:00:00', '2023-01-26 05:00:00'],
              dtype='datetime64[ns]', name='datetime', length=756114, freq=None)

In [45]:
assert (time_index.hour == new_consumption_train["hour"]).sum() == len(new_consumption_train)
assert (time_index.dayofweek == new_consumption_train["dayofweek"]).sum() == len(new_consumption_train)
assert (time_index.month == new_consumption_train["month"]).sum() == len(new_consumption_train)

# Pipeline

In [46]:
def feature_engineering_pipeline(df):
    start = config["forecast_horizon"]
    end = start + config["n_lags"] + 1
    groups = df.groupby(by="prediction_unit_id")
    transformed_groups = []
    for name, group in groups:
        group = make_lags(df=group, column="target", lags=range(start, end))
        transformed_groups.append(group)
    new_df = pd.concat(transformed_groups, axis=0)
    new_df.reset_index(drop=True, inplace=True)

    new_df = make_time_features(df=new_df, datetime_column="datetime")

    return new_df

In [47]:
df_ = feature_engineering_pipeline(consumption_train)

In [48]:
(df_ == new_consumption_train).sum().sum() == df_.size

np.True_

In [49]:
del df_, new_consumption_train

In [50]:
#TODO: stack the last 'n_lags' rows from train on top of test so that we keep all rows from test after processing
consumption_test = pd.read_csv("../data/preprocessed/consumption_test.csv", parse_dates=["datetime"])

In [51]:
consumption_train_processed = feature_engineering_pipeline(consumption_train)
consumption_test_processed = feature_engineering_pipeline(consumption_test)
print(consumption_train_processed.shape)
print(consumption_test_processed.shape)

(756114, 37)
(185598, 37)


# Save transformed data

In [55]:
import os

In [44]:
if "processed" not in os.listdir("../data"):
    os.mkdir("../data/processed")

In [52]:
# TODO: save in an other format to preserve data types
consumption_train_processed.to_csv("../data/processed/consumption_train_processed.csv", index=False)

In [53]:
consumption_test_processed.to_csv("../data/processed/consumption_test_processed.csv", index=False)

In [56]:
os.listdir("../data/processed")

['consumption_train_processed.csv', 'consumption_test_processed.csv']

In [ ]:
# Load data to AWS S3 (Optional)
if config["s3_bucket"]:
    s3_client = boto3.client('s3')
    with open("../data/processed/train.csv", "rb") as file:
        s3_client.upload_fileobj(file, config["s3_bucket"], "data/processed/train.csv")
    with open("../data/processed/test.csv", "rb") as file:
        s3_client.upload_fileobj(file, config["s3_bucket"], "data/processed/test.csv")

In [ ]:
# Check that data has been uploaded correctly
if config["s3_bucket"]:
    response = s3_client.list_objects(
        Bucket=config["s3_bucket"])
    for obj in response.get("Contents"):
        print(obj.get("Key"))